In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [144]:
import random
import sys
import numpy as np
import math
import matplotlib.pyplot as plt

In [160]:
population_size = 70
max_generations = 10
max_iter = 100
repeat = 3
outfile = sys.stdout
mut_rate = 0.25
stddev = 0.02
with open('/content/drive/MyDrive/fajl.txt', 'r') as f:
    for linija in f:
        if 'population_size' in linija:
            broj_string = linija.split('population_size = ')[1].strip()
            try:
                population_size = int(broj_string)
            except ValueError:
                print(f"Nije moguće pretvoriti {broj_string} u broj.")
        if 'max_generations' in linija:
            broj_string = linija.split('max_generations = ')[1].strip()
            try:
                max_generations = int(broj_string)
            except ValueError:
                print("Nije moguće pretvoriti u broj.")
        if 'max_iter' in linija:
            broj_string = linija.split('max_iter = ')[1].strip()
            try:
                max_iter = int(broj_string)
            except ValueError:
                print(f"Nije moguće pretvoriti {broj_string} u broj.")
        if 'repeat' in linija:
            broj_string = linija.split('repeat = ')[1].strip()
            try:
                repeat = int(broj_string)
            except ValueError:
                print(f"Nije moguće pretvoriti {broj_string} u broj.")
        if 'mut_rate' in linija:
            broj_string = linija.split('mut_rate = ')[1].strip()
            try:
                num_variables = int(broj_string)
            except ValueError:
                print(f"Nije moguće pretvoriti {broj_string} u broj.")
        if 'stddev' in linija:
            broj_string = linija.split('stddev = ')[1].strip()
            try:
                stddev = int(broj_string)
            except ValueError:
                print(f"Nije moguće pretvoriti {broj_string} u broj.")
        if 'outfile' in linija:
            outfile = linija.split('outfile = ')[1].strip()

In [146]:
import subprocess

def compile_c_program():
    compile_args = ['gcc', '-Wall', '/content/drive/MyDrive/g.c', '-o', 'k', '-lm']
    subprocess.call(compile_args)

compile_c_program()

In [147]:
def loss(niz_koeficijenata):

    string_array = [str(float_value) for float_value in niz_koeficijenata]
    #print(len(niz_koeficijenata))

    output = subprocess.check_output(["/content/drive/MyDrive/k", *string_array], universal_newlines=True)

    result = float(output.strip())

    return result

In [148]:
# Funkcija za mutaciju (tačkasta normalna mutacija)
def mutate (hromozom, rate):
    if random.random() < rate:
        for i in range(33):
            hromozom[i] += random.gauss(0, stddev)

    return hromozom

In [149]:
# Funkcija za generisanje inicijalne populacije
def initialize_population(population_size):
    population =[]
    for i in range(population_size):
            niz_koef = []
            for j in range(33):
                niz_koef.append(round(random.uniform(-3,3), 3))
            population.append(niz_koef)

    return population

# Funkcija za selekciju roditelja (proporcionalna selekcija)
def select_parents(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [score / total_fitness for score in fitness_scores]
    selected_index = random.choices(range(len(population)), weights=probabilities, k=1)[0]
    return population[selected_index]

# Funkcija za ukrštanje (heurističko ukrštanje)
def crossover(parent1, parent2):
    #print('ukrstanje')
    child1 = []
    child2 = []
    #print(parent1)
    for gene1, gene2 in zip(parent1, parent2):
        alpha = random.uniform(0, 1)
        child1_gene = alpha * gene1 + (1 - alpha) * gene2
        child2_gene = alpha * gene2 + (1 - alpha) * gene1
        child1.append(child1_gene)
        child2.append(child2_gene)

    return child1, child2

In [158]:
def cga():
  open()
  population_size = 40
  npop_vel = population_size
  c = 0
  s_trosak = 0
  s_iteracija = 0
  best_ever_sol = None
  best_ever_f = None
  fig1 = plt.figure(figsize=(20, 4))
  ax1 = fig1.add_subplot(1,2,1)
  ax1.set_xlabel('редни број генерације')
  ax1.set_ylabel('вр ф за нај реш у одређеном покретању у тренутној ген')
  fig1.suptitle('График прилагођености најбољег решења у популацији у односу на редни број генерације')
  fig2 = plt.figure(figsize=(20, 4))
  fig2.suptitle('График просечне прилагођености популације у односу на редни број генерације')
  ax2 = fig2.add_subplot(1,2,1)
  ax2.set_xlabel('редни број генерације')
  ax2.set_ylabel('вр ф за нај реш у одређеном покретању у тренутној ген')
  # 3 pokretanja genetskog algoritma
  for k in range(repeat):
        xvalues=[]
        yvalues=[]
        yvalues1=[]
        print('Pokretanje: GA', mut_rate, k, file=outfile)
        best = None
        best_f = None
        t = 0
        #print('petlja1')
        pop = initialize_population(population_size)
        #print(pop)
        fitness_scores = [loss(population) for population in pop]
        # ponavljamo dok ne postignemo maksimum iteracija ili dok trošak ne postane 0
        while best_f != 0 and t < max_iter:
            #print('petlja2')
            n_pop = pop[:]
            i=0
            svi_troskovi=0
            while len(n_pop) < population_size + npop_vel:
                #print('petlja3')
                h1 = select_parents(pop, fitness_scores)
                h2 = select_parents(pop, fitness_scores)
                h3, h4 = crossover(h1, h2)
                #print(h3)
                mutate(h3, mut_rate)
                mutate(h4, mut_rate)
                n_pop.append(h3)
                n_pop.append(h4)
                svi_troskovi+=loss(pop[i])
                i+=1
            pop = sorted(n_pop, key=lambda x : loss(x))[:population_size]
            fitness_scores = [loss(population) for population in pop]
            f = loss(pop[0])
            if best_f is None or best_f > f:
                best_f = f
                best = pop[0]
            xvalues.append(t)
            t += 1
            yvalues.append(f)
            yvalues1.append(svi_troskovi/population_size)
        s_trosak += best_f
        s_iteracija += t

        if best_ever_f is None or best_ever_f > best_f:
            best_ever_f = best_f
            best_ever_sol = best
        print(t, best, best_f, file=outfile)
        ax1.plot(xvalues, yvalues, label=k)
        ax1.legend()
        ax2.plot(xvalues, yvalues1, label=k)
        ax2.legend()
    # na kraju svih izvršavanja izračunavamo srednji trošak i srednji broj iteracija
  s_trosak /= 3
  s_iteracija /= 3
  print('Srednji trosak: %.2f' % s_trosak, file=outfile)
  print('Srednji broj iteracija: %.2f' % s_iteracija, file=outfile)
  print('Najbolje resenje: %s' % best_ever_sol, file=outfile)
  print('Najbolji trosak: %.2f' % best_ever_f, file=outfile)


In [ ]:
cga()